In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import rotation_matrix as rm

In [2]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [3]:
rtde_c, rtde_r, rtde_io_set = connect_robot()

In [66]:
print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(10)  # from 0 to 100 %
gripper.set_speed(10)  # from 0 to 100 %
print("Gripper activated")

Activating Gripper
Gripper activated


In [4]:
import numpy as np

data = np.loadtxt('../pouring_simulation/output/Assembly1_TCP.txt', delimiter=',', skiprows=1)

In [5]:
data.shape

(981, 3)

In [6]:
data[500]

# convert from inches to meters
data[:, 0] = data[:, 0] * 0.0254
data[:, 1] = data[:, 1] * 0.0254

In [32]:
data[500]

array([0.01993362, 0.07887081, 0.496261  ])

In [72]:
start_joints = [3.127164125442505,
 -1.275574342613556,
 -2.3293192386627197,
 -1.4077772957137604,
 0.020992517471313477,
 0.3164108395576477]

start_pos = [-0.31755315085470437,
 0.3776415807801057,
 0.3636001383133041,
 -1.2357319406800302,
 -1.196658279424475,
 -1.204508997154227]

rtde_c.moveJ(start_joints, 0.1, 0.1)

True

In [9]:
pos2 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
pos2[1] -= data[500, 0] # move x axis of tool
pos2[0] += data[500, 1] # move y axis of tool
pos2[5] -= data[500, 2] # rotate z axis of tool

In [69]:
new_pos = rm.PoseTrans(start_pos, pos2) # transform from base to tool coordinates

In [70]:
new_pos

[-0.33619875534446625,
 0.37871109474915343,
 0.4427781103641161,
 -0.9625305855867964,
 -1.55825789481876,
 -1.558227306204526]

In [71]:
rtde_c.moveL(new_pos, 0.1, 0.1)

True

In [11]:
current_pos = rtde_r.getActualTCPPose()

current_pos


[-0.31755315085470437,
 0.3776415807801057,
 0.3636001383133041,
 -1.2357319406800302,
 -1.196658279424475,
 -1.204508997154227]

In [12]:
init_q = rtde_r.getActualQ()
init_q

[3.127164125442505,
 -1.275574342613556,
 -2.3293192386627197,
 -1.4077772957137604,
 0.020992517471313477,
 0.3164108395576477]

In [10]:
rtde_c.moveJ(init_q, 0.1, 0.2)

False

In [ ]:
velocity = 0.1
acceleration = 0.1
blend_1 = 0.0
blend_2 = 0.02
blend_3 = 0.0
path_pose1 = [-0.143, -0.435, 0.20, -0.001, 3.12, 0.04, velocity, acceleration, blend_1]
path_pose2 = [-0.143, -0.51, 0.21, -0.001, 3.12, 0.04, velocity, acceleration, blend_2]
path_pose3 = [-0.32, -0.61, 0.31, -0.001, 3.12, 0.04, velocity, acceleration, blend_3]
path = [path_pose1, path_pose2, path_pose3]

# Send a linear path with blending in between - (currently uses separate script)
rtde_c.moveL(path)
rtde_c.stopScript()

In [7]:
start_pos = rtde_r.getActualTCPPose()

start_pos

[-0.32840330749723673,
 0.6906975268273259,
 0.25610656751907307,
 -1.2222948307378443,
 -1.1666822479428505,
 -1.218172452957232]

In [10]:
new_pos = rm.PoseTrans(start_pos, pos2) # transform from base to tool coordinates

In [11]:
rtde_c.moveL(new_pos, 0.1, 0.1)

True